In [49]:
import networkExpansionPy.lib as ne
from scipy.sparse import csr_matrix
import numpy as np
from random import sample
import pandas as pd

#from copy import copy, deepcopy
import random
import seaborn as sns
from matplotlib_venn import venn2
import matplotlib.pyplot as plt


In [50]:
metabolism = ne.GlobalMetabolicNetwork()
metabolism.set_ph(7.0)
metabolism.pruneUnbalancedReactions()
metabolism.pruneInconsistentReactions()


In [51]:
metabolism.addGenericCoenzymes()

In [52]:
# prune o2 dependent reactions
oxygen_dependent_rxns = metabolism.network[metabolism.network.cid.isin(['C00007'])].rn.unique().tolist()
o2_independent_rxns = [x for x in metabolism.network.rn.unique().tolist() if x not in oxygen_dependent_rxns]
# only keep anaerobic reactions
metabolism.subnetwork(o2_independent_rxns)

In [53]:
# prune reactions using iron or mn only

# load EC3 specificty mapping from metal
rxns = pd.read_csv('../../networkExpansionPy/networkExpansionPy/assets/reaction_sets/reactions.FeMnCatalyzed.01Feb2021.csv')
# load EC4 specificity mapping from EC to reaction
#file_id = 'reactions.EC4.FeMnCatalyzed.15Feb2021.csv'
#rxns = pd.read_csv('../../networkExpansionPy/networkExpansionPy/assets/reaction_sets/' + file_id)

# include generic reactions with NAD/FAD coupling
rxns = pd.concat([rxns[['rn']],pd.DataFrame(rxns['rn'] + '_G')],axis=0)
rxns = rxns['rn'].tolist()

rxn_all = metabolism.network.rn.unique().tolist()
rxns = [r for r in rxns if r in rxn_all]

pan_deg = ['R02973','R10747','R02972']
pan_deg = pan_deg + [x + '_G' for x in pan_deg]
rxns = [r for r in rxns if r not in pan_deg]
print('trimming network to ' + str(len(rxns)) + ' reactions')

metabolism.subnetwork(rxns)

trimming network to 6392 reactions


In [54]:
metabolism.convertToIrreversible()
#metabolism.convertToIrreversible()

In [55]:
metabolism.setMetaboliteBounds(ub=1e-1,lb=1e-6)
metabolism.pruneThermodynamicallyInfeasibleReactions(keepnan=False)

In [56]:
#
#rxns_fe_mn= pd.read_csv('../../networkExpansionPy/networkExpansionPy/assets/reaction_sets/reactions.FeMnCatalyzed.01Feb2021.csv')
len(rxns)

6392

In [44]:
#metabolism.network

In [60]:
#metabolism.network
#seed_set = set(['C00001','C00011','C00022','C00033','C00048','C00282','C00010','Generic_oxidant'])
seed_set = set(['C00001','C00011','C00282','C00283','C00010','Generic_oxidant'])
cpds,rxns_new = metabolism.expand(list(seed_set))

/Users/joshuagoldford/Documents/github/networkExpansionPy/networkExpansionPy/lib.py:236: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  x,y = netExp(R,P,x0,b)


In [61]:
len(cpds)

190

In [28]:
metabolism.compounds[metabolism.compounds.cid.isin(cpds)].iloc[201:250]

,cid,formula,name,num_rgroup,electrons
3880,C05261,C35H60N7O18P3S,3-Oxotetradecanoyl-CoA,0,524.0
3881,C05262,C33H58N7O18P3S,(S)-3-Hydroxydodecanoyl-CoA,0,510.0
3882,C05263,C33H56N7O18P3S,3-Oxododecanoyl-CoA,0,508.0
3883,C05264,C31H54N7O18P3S,(S)-Hydroxydecanoyl-CoA; (S)-3-Hydroxydecanoyl...,0,494.0
3884,C05265,C31H52N7O18P3S,3-Oxodecanoyl-CoA,0,492.0
3885,C05266,C29H50N7O18P3S,(S)-3-Hydroxyoctanoyl-CoA; (S)-3-Hydroxycapryl...,0,478.0
3886,C05267,C29H48N7O18P3S,3-Oxooctanoyl-CoA,0,476.0
3887,C05268,C27H46N7O18P3S,(S)-Hydroxyhexanoyl-CoA; (S)-3-Hydroxyhexanoyl...,0,462.0
3888,C05269,C27H44N7O18P3S,3-Oxohexanoyl-CoA; 3-Ketohexanoyl-CoA,0,460.0
3889,C05270,C27H46N7O17P3S,Hexanoyl-CoA,0,454.0


In [ ]:
metabolism.pruneUnbalancedReactions()
metabolism.pruneInconsistentReactions()
# prune reactions using iron or mn only
rxns = pd.read_csv('../../networkExpansionPy/networkExpansionPy/assets/reaction_sets/reactions.FeMnCatalyzed.01Feb2021.csv')
# include generic reactions with NAD/FAD coupling
rxns = pd.concat([rxns[['rn']],pd.DataFrame(rxns['rn'] + '_G')],axis=0)
rxns = rxns['rn'].tolist()

metabolism.subnetwork(rxns)
metabolism.convertToIrreversible()

oxygen_dependent_rxns = metabolism.network[metabolism.network.cid.isin(['C00007'])].rn.unique().tolist()
o2_independent_rxns = [x for x in metabolism.network.rn.unique().tolist() if x not in oxygen_dependent_rxns]
# only keep anaerobic reactions
metabolism.subnetwork(o2_independent_rxns)
metabolism.setMetaboliteBounds(ub=1e-1,lb=1e-6)
metabolism.pruneThermodynamicallyInfeasibleReactions(keepnan=False)

In [22]:
rxns = pd.read_csv('../../networkExpansionPy/networkExpansionPy/assets/reaction_sets/reactions.FeMnCatalyzed.01Feb2021.csv')
# include generic reactions with NAD/FAD coupling
rxns = pd.concat([rxns[['rn']],pd.DataFrame(rxns['rn'] + '_G')],axis=0)

In [24]:
rxns = rxns['rn'].tolist()

In [30]:
any([x[-1] == 'G' for x in rxns])

True

In [38]:
seed_set = set(['C00001','C00011','C00022','C00033','C00048','C00282'])

In [39]:
cpds,rxns_new = metabolism.expand(list(seed_set))

/Users/joshuagoldford/Documents/github/networkExpansionPy/networkExpansionPy/lib.py:204: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  x,y = netExp(R,P,x0,b)


In [40]:
len(rxns_new)

71

In [41]:
metabolism.compounds[metabolism.compounds.cid.isin(cpds)]

,cid,formula,name,num_rgroup,electrons
0,C00001,H2O,H2O; Water,0,10.0
10,C00011,CO2,CO2; Carbon dioxide,0,22.0
21,C00022,C3H4O3,Pyruvate; Pyruvic acid; 2-Oxopropanoate; 2-Oxo...,0,46.0
32,C00033,C2H4O2,Acetate; Acetic acid; Ethanoic acid,0,32.0
35,C00036,C4H4O5,Oxaloacetate; Oxalacetic acid; Oxaloacetic aci...,0,68.0
41,C00042,C4H6O4,Succinate; Succinic acid; Butanedionic acid; E...,0,62.0
47,C00048,C2H2O3,Glyoxylate; Glyoxalate; Glyoxylic acid,0,38.0
81,C00084,C2H4O,Acetaldehyde; Ethanal,0,24.0
153,C00158,C6H8O7,"Citrate; Citric acid; 2-Hydroxy-1,2,3-propanet...",0,100.0
163,C00168,C3H4O4,Hydroxypyruvate; Hydroxypyruvic acid; 3-Hydrox...,0,54.0


In [43]:
network = pd.read_csv('../../networkExpansionPy/networkExpansionPy/assets/KEGG/network_full.csv')


In [74]:

def isRxnCoenzymeCoupled(rxn,cosubstrate,coproduct):
    g = rxn[rxn.cid.isin([cosubstrate,coproduct])]
    out = False
    if len(g) > 1:
        if g.s.sum() == 0:
            out = True
    return out

In [104]:
replace_metabolites = {'C00003': 'Generic_oxidant', 'C00004': 'Generic_reductant', 'C00006': 'Generic_oxidant',  'C00005': 'Generic_reductant','C00016': 'Generic_oxidant','C01352':'Generic_reductant'}
coenzyme_pairs = {}
coenzyme_pairs['NAD'] = ['C00003','C00004']
coenzyme_pairs['NADP'] = ['C00006','C00005']
coenzyme_pairs['FAD'] = ['C00016','C01352']
coenzyme_pairs = pd.DataFrame(coenzyme_pairs).T.reset_index()
coenzyme_pairs.columns = ['id','oxidant','reductant']

# create reactions copies with coenzyme pairs
new_rxns = []
new_thermo = [];
for idx,rxn in network.groupby('rn'):
    z = any([isRxnCoenzymeCoupled(rxn,row.oxidant,row.reductant) for x,row in coenzyme_pairs.iterrows()])
    if z:
        new_rxn = rxn.replace(replace_metabolites).groupby(['cid','rn']).sum().reset_index()
        new_rxn['rn'] = new_rxn['rn'] = idx + '_G'
        new_rxns.append(new_rxn)
        t = metabolism.thermo[metabolism.thermo['!MiriamID::urn:miriam:kegg.reaction'] == idx].replace({idx:  idx + '_G'})
        new_thermo.append(t)
        
new_rxns = pd.concat(new_rxns,axis=0)
new_thermo = pd.concat(new_thermo,axis=0)

self.network = pd.concat([self.network,new_rxns],axis=0)
self.thermo = pd.concat([self.thermo,new_thermo],axis=0)

In [107]:
new_rxns = pd.concat(new_rxns,axis=0)
new_thermo = pd.concat(new_thermo,axis=0)

In [113]:
pd.concat([network,new_rxns],axis=0)

,cid,rn,s
0,C00001,R00002,-16
1,C00002,R00002,-16
2,C00008,R00002,16
3,C00009,R00002,16
4,C00138,R00002,-8
...,...,...,...
2,C00080,R10879_G,-1
3,C18845,R10879_G,1
4,C18846,R10879_G,-1
5,Generic_oxidant,R10879_G,1


In [103]:
t = metabolism.thermo[metabolism.thermo['!MiriamID::urn:miriam:kegg.reaction'] == idx].replace({idx:  idx + '_G'})

,!MiriamID::urn:miriam:kegg.reaction,!dG0_prime (kJ/mol),!sigma[dG0] (kJ/mol),!pH,!I (mM),!T (Kelvin),!Note
18,R00023_G,NaN,NaN,7.0,0.1,298.15,uncertainty is too high


In [98]:
idx

'R10879'